In [1]:
# %pip install pyscisci networkx python-Levenshtein sparse_dot_topn matplotlib boto3 widgetsnbextension ipywidgets "dask[distributed]<2024.3.0"  --upgrade


In [2]:
import pyscisci.all as pyscisci

import random
import pickle
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import dask.dataframe as dd

%matplotlib inline

path2openalex = '/disks/qnap2/data/t-yatima/openalex231124/data/'

DATA_DIR = '/disks/qnap2/data/t-yatima/scisciJP2024_tutorial/'
INPUT_D_LIST_PKL = DATA_DIR + 'D_values.pkl'
INPUT_WU_WORKS_LIST_PKL = DATA_DIR + 'teamsize_impact_disrup_refAge_refPop.pkl'
OUTPUT_WU_WORKS_LIST_SMALL_PKL = 'supplementary/teamsize_impact_disrup_refAge_refPop_small.pkl'
OUTPUT_WORKS_DF_PKL = DATA_DIR + 'random_works_df.pkl'
OUTPUT_WORKS_SMALL_DF_PKL = 'supplementary/random_works_small_df.pkl'
OUTPUT_WORKS_REL_REF_DF_PKL = DATA_DIR + 'random_rel_ref_df.pkl'
OUTPUT_WORKS_REL_REF_SMALL_DF_PKL = 'supplementary/random_rel_ref_small_df.pkl'
# OUTPUT_CITATIONS_GB_DF_PKL = DATA_DIR + 'citations_gb_df.pkl'

myoa = pyscisci.OpenAlex(path2openalex, database_extension='csv.gz', keep_in_memory=True, show_progress=True, enable_dask=True) 

from dask.diagnostics import ProgressBar
ProgressBar().register()

from dask.distributed import progress

In [3]:
all_pub_ddf = myoa.load_publications()
all_pub_ddf

Loading Publications:   0%|          | 0/480 [00:00<?, ?it/s]

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
npartitions=480,,,,,,,,,,,,,,,
,string,string,string,string,string,string,string,string,string,string,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
# 9min
references_df = myoa.load_references()
display(references_df)

Loading pub2ref:   0%|          | 0/480 [00:00<?, ?it/s]

,CitingPublicationId,CitedPublicationId
0,3117235547,55183993
1,3117235547,657382535
2,3117235547,1540739244
3,3117235547,1582266453
4,3117235547,1642177666
...,...,...
9436475,2134809571,2090594024
9436476,2134809571,2124054840
9436477,2134809571,2147932722
9436478,2134809571,2158145285


In [5]:
references_df[references_df['CitingPublicationId'] == 2133895471]

,CitingPublicationId,CitedPublicationId


In [6]:
len(references_df[references_df['CitedPublicationId'] == 2133895471])

50

In [7]:
# 8min
random_works_ddf = all_pub_ddf.sample(frac=0.000005, random_state=1)
random_works_df = random_works_ddf.compute()

[                                        ] | 1% Completed | 1.10 s ms

/home/t-yatima/t-yatima/.venv/lib/python3.10/site-packages/dask/dataframe/backends.py:688: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat(dfs3, join=join, sort=False)


[########################################] | 100% Completed | 16m 12s


In [8]:
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
84767,2242094754,<NA>,2005,0,Understanding what child protection data mean:...,2005-08-11,article,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
183235,2951317825,<NA>,2015,0,Desvendando o Universo com o Céu da Serra Gaúcha,2015-11-30,article,<NA>,<NA>,3,3,<NA>,<NA>,False,False
11528,2170217184,<NA>,2006,50,Novel approach to fabricating carbon nanotube ...,2006-01-01,article,10.1109/iitc.2006.1648696,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
51663,2332852995,<NA>,1942,18,Form und Ausdehnung der Panum schen Areale bei...,1942-10-01,article,10.1007/bf02562544,<NA>,246,2,315,328,False,False
281880,2027815162,<NA>,1999,2,Selected ideas of the theory of non‐linear ele...,1999-06-01,article,10.1108/03321649910264208,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259162,2129436324,<NA>,1999,44,Bone health: the role of micronutrients,1999-01-01,article,10.1258/0007142991902501,10746351,55,3,619,633,False,False
176926,2090206302,<NA>,1980,36,CLINICAL TRIAL OF DESFERRIOXAMINE ENTRAPPED IN...,1980-08-01,article,10.1016/s0140-6736(80)90336-0,6105474,316,8190,327,330,False,False
284765,2069294162,<NA>,1996,33,Nonvisualization of the fetal gallbladder: fre...,1996-06-01,article,10.1148/radiology.199.3.8637986,8637986,199,3,679,682,False,False
149535,3167557254,<NA>,2021,29,C-type lectins and extracellular vesicles in v...,2021-06-11,article,10.1186/s12929-021-00741-7,34116654,28,1,<NA>,<NA>,False,False


In [9]:
display(random_works_df[random_works_df['PublicationId'] == 1982793646], random_works_df[random_works_df['PublicationId'] == 2127548443], random_works_df[random_works_df['PublicationId'] == 2170866237])

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText


,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText


,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText


In [10]:
# ネットワークの可視化 + DNA/RNA用
selected_df = all_pub_ddf[all_pub_ddf['PublicationId'].isin([1982793646, 2127548443, 2170866237, 2126466006, 2013241878])].compute()
selected_df

[########################################] | 100% Completed | 16m 10s


,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText


In [11]:
random_works_df = pd.concat([selected_df, random_works_df])
random_works_df

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText
84767,2242094754,<NA>,2005,0,Understanding what child protection data mean:...,2005-08-11,article,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
183235,2951317825,<NA>,2015,0,Desvendando o Universo com o Céu da Serra Gaúcha,2015-11-30,article,<NA>,<NA>,3,3,<NA>,<NA>,False,False
11528,2170217184,<NA>,2006,50,Novel approach to fabricating carbon nanotube ...,2006-01-01,article,10.1109/iitc.2006.1648696,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
51663,2332852995,<NA>,1942,18,Form und Ausdehnung der Panum schen Areale bei...,1942-10-01,article,10.1007/bf02562544,<NA>,246,2,315,328,False,False
281880,2027815162,<NA>,1999,2,Selected ideas of the theory of non‐linear ele...,1999-06-01,article,10.1108/03321649910264208,<NA>,<NA>,<NA>,<NA>,<NA>,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259162,2129436324,<NA>,1999,44,Bone health: the role of micronutrients,1999-01-01,article,10.1258/0007142991902501,10746351,55,3,619,633,False,False
176926,2090206302,<NA>,1980,36,CLINICAL TRIAL OF DESFERRIOXAMINE ENTRAPPED IN...,1980-08-01,article,10.1016/s0140-6736(80)90336-0,6105474,316,8190,327,330,False,False
284765,2069294162,<NA>,1996,33,Nonvisualization of the fetal gallbladder: fre...,1996-06-01,article,10.1148/radiology.199.3.8637986,8637986,199,3,679,682,False,False
149535,3167557254,<NA>,2021,29,C-type lectins and extracellular vesicles in v...,2021-06-11,article,10.1186/s12929-021-00741-7,34116654,28,1,<NA>,<NA>,False,False


In [12]:
# 11min
all_paa = myoa.load_publicationauthoraffiliation()
all_paa

Loading Publication Author Affiliation:   0%|          | 0/480 [00:00<?, ?it/s]

,PublicationId,AuthorId,AffiliationId,AuthorSequence,OrigAuthorName
0,4287754650,5062226449,NaN,1,Enrico Biffis
1,4287754650,5033140346,NaN,2,Erik Chavez
2,4287754650,5061623148,NaN,3,Alexis Louaas
3,4287754650,5074952038,NaN,4,Pierre M. Picard
4,4287938432,5045351019,NaN,1,Arthur Hugeat
...,...,...,...,...,...
1810902,2134809571,5004197195,1.355989e+08,3,Hiroki Shibata
1810903,2134809571,5061432587,4.210123e+09,4,Tomomi Saito
1810904,2134809571,5029531503,4.240550e+07,5,Osamu Doi
1810905,2134809571,5004358248,4.240550e+07,6,Yutaka Ohya


In [13]:
all_paa[all_paa['PublicationId'].isin([653597189, 2242094754, 2951317825])]

,PublicationId,AuthorId,AffiliationId,AuthorSequence,OrigAuthorName
154625,2242094754,5026037360,NaN,1,Daryl Higgins
154626,2242094754,5053282767,NaN,2,Leah Bromfield
331280,2951317825,5005072967,2.801050e+09,1,Caroline Azzolini Pontel
331281,2951317825,5078242247,2.801050e+09,2,Thailise de Oliveira Candido
331282,2951317825,5083600524,2.801050e+09,3,Leonardo Kramer Pezzin
331283,2951317825,5018366926,2.801050e+09,4,Marla Heckler


In [14]:
all_paa[all_paa['PublicationId'] == 2242094754]

,PublicationId,AuthorId,AffiliationId,AuthorSequence,OrigAuthorName
154625,2242094754,5026037360,NaN,1,Daryl Higgins
154626,2242094754,5053282767,NaN,2,Leah Bromfield


In [15]:
random_works_df['PublicationId'].to_list()

['2242094754',
 '2951317825',
 '2170217184',
 '2332852995',
 '2027815162',
 '1830373157',
 '1429986587',
 '2004061347',
 '2293869977',
 '2377169427',
 '2370499641',
 '2350967676',
 '2391211130',
 '2380284986',
 '2401942128',
 '2467254540',
 '3134187599',
 '2748061084',
 '4291605330',
 '2622370393',
 '884281907',
 '4205279783',
 '1842007641',
 '2548610616',
 '2775296517',
 '2802440950',
 '2333663200',
 '2738560831',
 '2913111331',
 '2998666130',
 '2990399118',
 '2919675454',
 '3203356558',
 '3122060510',
 '3113062445',
 '397795794',
 '953577119',
 '625197289',
 '1564828837',
 '2161194498',
 '2224838514',
 '2625424264',
 '2061064219',
 '1010664123',
 '2437433125',
 '1552517461',
 '1562370559',
 '1826541919',
 '1563287870',
 '102365665',
 '223684252',
 '2253442891',
 '2082304819',
 '186505441',
 '2276223657',
 '2359845262',
 '2312867456',
 '2347451845',
 '2369773757',
 '2372680630',
 '2390526426',
 '2527162214',
 '2593858188',
 '2524609854',
 '2588666659',
 '2656584671',
 '2914234744',
 '

In [ ]:
id_list = random_works_df['PublicationId'].to_list()
print(len(id_list))
needed_paa = all_paa[all_paa['PublicationId'].isin(id_list)]
print(len(needed_paa))
author_count_list = needed_paa.groupby('PublicationId').agg(AuthorCount=('AuthorId', 'count'), AuthorNames=('OrigAuthorName', list)).reset_index()
print(len(author_count_list))
random_works_df = random_works_df.merge(author_count_list, on='PublicationId', how='left')
random_works_df

In [26]:
random_works_df[random_works_df['AuthorCount'].isna()]

,PublicationId,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
26,4205279783,NaN,1977,1,Spotlight,1977-01-01,article,10.1080/03060497.1977.11083557,NaN,5,5,141,141,False,False,NaN
371,4250560449,NaN,1995,0,Head trauma and blood diseases,1995-05-01,article,10.1007/bf02753590,NaN,62,3,291,291,False,False,NaN
423,4383459494,NaN,1991,0,Statistical Summary of the Rural System in Sou...,1991-12-05,book-chapter,10.1515/9780773563100-017,NaN,NaN,NaN,266,267,False,False,NaN
427,4384636507,NaN,2023,0,"physa, n.",2023-03-02,book-chapter,10.1093/oed/1955218699,NaN,NaN,NaN,NaN,NaN,False,False,NaN
501,4254714524,NaN,1939,1,Dominations and Elections,1939-12-01,article,10.5694/j.1326-5377.1939.tb115614.x,NaN,2,23,852,852,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148,4251681702,NaN,2000,0,CONTENTS,2000-01-01,paratext,10.1515/revneuro.2000.11.2-3.i,NaN,11,2-3,NaN,NaN,False,True,NaN
1149,4250212770,NaN,2021,0,11. Sabdapramana,2021-01-01,article,10.2139/ssrn.3941780,NaN,NaN,NaN,NaN,NaN,False,False,NaN
1150,4247671114,NaN,1969,0,THERAPIST ORIENTATION QUESTIONNAIRE,1969-01-01,book-chapter,10.1016/b978-0-08-006422-2.50024-2,NaN,NaN,NaN,184,192,False,False,NaN
1151,4247704377,NaN,1998,0,Sociolinguistics,1998-04-01,article,10.1017/s0261444800013057,NaN,31,2,106,109,False,False,NaN


In [28]:
all_paa[all_paa['PublicationId'].isin([4255374622, 4247704377, 4247671114])]

,PublicationId,AuthorId


In [29]:
bibmet_id_list = [int(work) for work in random_works_df['PublicationId'].to_list()]
bibmet_id_list

[2170866237,
 1982793646,
 2013241878,
 2126466006,
 2127548443,
 2242094754,
 2951317825,
 2170217184,
 2332852995,
 2027815162,
 1830373157,
 1429986587,
 2004061347,
 2293869977,
 2377169427,
 2370499641,
 2350967676,
 2391211130,
 2380284986,
 2401942128,
 2467254540,
 3134187599,
 2748061084,
 4291605330,
 2622370393,
 884281907,
 4205279783,
 1842007641,
 2548610616,
 2775296517,
 2802440950,
 2333663200,
 2738560831,
 2913111331,
 2998666130,
 2990399118,
 2919675454,
 3203356558,
 3122060510,
 3113062445,
 397795794,
 953577119,
 625197289,
 1564828837,
 2161194498,
 2224838514,
 2625424264,
 2061064219,
 1010664123,
 2437433125,
 1552517461,
 1562370559,
 1826541919,
 1563287870,
 102365665,
 223684252,
 2253442891,
 2082304819,
 186505441,
 2276223657,
 2359845262,
 2312867456,
 2347451845,
 2369773757,
 2372680630,
 2390526426,
 2527162214,
 2593858188,
 2524609854,
 2588666659,
 2656584671,
 2914234744,
 2742997455,
 2745055512,
 2741413905,
 2914737786,
 2997360397,
 30100

In [30]:
# 6min
# (B) 引用先の論文の抽出
B_relations = references_df[references_df['CitedPublicationId'].isin(bibmet_id_list)]

# (C) 参考文献に挙げた論文
C_relations = references_df[references_df['CitingPublicationId'].isin(bibmet_id_list)]

# (D) 更に「参考文献に挙げた論文」を参考文献に挙げている論文の抽出
# まず(C)からCitedPublicationIdを取得し、それを引用している論文を探す
C_cited_ids = C_relations['CitedPublicationId'].unique()
D_relations = references_df[references_df['CitedPublicationId'].isin(C_cited_ids)]

B_relations, D_relations

(        CitingPublicationId CitedPublicationId
 15179            2082805732         2069814472
 61785            1461953811         2093032015
 164653           3177720207         2126466006
 191593           1605452458         2126466006
 212078           2086215418         2126466006
 ...                     ...                ...
 9294566          2081093505         2126466006
 9306233          2084205412         2013241878
 9334559          2091687374         1970634720
 9375439          2107422072         2013241878
 9394085          2115749521         2013241878
 
 [20124 rows x 2 columns],
         CitingPublicationId CitedPublicationId
 36               2093406744         2136022845
 567              2171641398         2107277218
 606              2136902758         2024112877
 257              1993661162         1775749144
 144              2119925004         2051921020
 ...                     ...                ...
 9434822          2134000226         2059542112
 9435652   

In [12]:
# D_relations[D_relations['CitedPublicationId'] == 1976880180]

,CitingPublicationId,CitedPublicationId


In [13]:
# D_relations[D_relations['CitingPublicationId'] == 2573593301]

,CitingPublicationId,CitedPublicationId
359,2573593301,2145007605


In [14]:
# references_df[references_df['CitedPublicationId'] == 55183993]

,CitingPublicationId,CitedPublicationId
0,3117235547,55183993
0,2031878785,55183993
3247108,1914197438,55183993
454530,3138296954,55183993
271815,648156390,55183993
...,...,...
953090,2460761523,55183993
6128321,585472863,55183993
6986859,2044549312,55183993
2359164,3195613568,55183993


In [15]:
# references_df[references_df['CitedPublicationId'] == '55183993']

,CitingPublicationId,CitedPublicationId


In [31]:
works_rel_ref_df = pd.concat([B_relations, C_relations, D_relations]).drop_duplicates().reset_index(drop=True)
works_rel_ref_df.rename(columns={'CitingPublicationId': 'source', 'CitedPublicationId': 'target'}, inplace=True)
works_rel_ref_df

,source,target
0,2082805732,2069814472
1,1461953811,2093032015
2,3177720207,2126466006
3,1605452458,2126466006
4,2086215418,2126466006
...,...,...
4882913,2134000226,2059542112
4882914,2134334595,2158714788
4882915,2134372864,1775749144
4882916,2134372864,4293247451


In [32]:
random_works_df.set_index('PublicationId', inplace=True)
random_works_df

,JournalId,Year,NumberCitations,Title,Date,DocType,Doi,PMID,Volume,Issue,FirstPage,LastPage,IsRetracted,IsParaText,AuthorCount
PublicationId,,,,,,,,,,,,,,,
2170866237,NaN,1988,2,Testing Safety Assessment Models Using Natural...,1988-01-01,article,10.1557/proc-127-863,NaN,NaN,NaN,NaN,NaN,False,False,5.0
1982793646,NaN,1989,7,Design Procedures for Fiber Composite Box Beams,1989-07-01,article,10.1177/073168448900800404,NaN,8,4,370,397,False,False,2.0
2013241878,NaN,1970,1900,Viral RNA-dependent DNA Polymerase: RNA-depend...,1970-06-01,article,10.1038/2261209a0,4316300,226,5252,1209,1211,False,False,1.0
2126466006,NaN,1953,11079,Molecular Structure of Nucleic Acids: A Struct...,1953-04-25,article,10.1038/171737a0,13054692,171,4356,737,738,False,False,2.0
2127548443,NaN,2015,2,An X-ray diffractometer using mirage diffracti...,2015-01-30,article,10.1107/s1600576714026028,26089753,48,1,312,312,False,False,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2129436324,NaN,1999,44,Bone health: the role of micronutrients,1999-01-01,article,10.1258/0007142991902501,10746351,55,3,619,633,False,False,1.0
2090206302,NaN,1980,36,CLINICAL TRIAL OF DESFERRIOXAMINE ENTRAPPED IN...,1980-08-01,article,10.1016/s0140-6736(80)90336-0,6105474,316,8190,327,330,False,False,6.0
2069294162,NaN,1996,33,Nonvisualization of the fetal gallbladder: fre...,1996-06-01,article,10.1148/radiology.199.3.8637986,8637986,199,3,679,682,False,False,8.0


In [37]:
random_works_df.to_pickle(OUTPUT_WORKS_SMALL_DF_PKL)
works_rel_ref_df.to_pickle(OUTPUT_WORKS_REL_REF_SMALL_DF_PKL)

In [8]:
works_list = pickle.load(open(INPUT_WU_WORKS_LIST_PKL, 'rb'))

In [27]:
works_list = random.sample(works_list,1000000)
display(len(works_list), works_list[:5])

1000000

[[7, 1, 0.0005817335660267597, 8.0, 114.0],
 [3, 2, -0.000461361014994233, 15.0, 234.5],
 [3, 254, -0.021009362868234755, 3.77419354839, 117.0],
 [2, 15, -0.004055619930475087, 6.66666666667, 42.0],
 [12, 6, -0.0020931449502878076, 5.24242424242, 15.0]]

In [34]:
pickle.dump(works_list, open(OUTPUT_WU_WORKS_LIST_SMALL_PKL, 'wb'))